https://facebook.github.io/prophet/docs/quick_start.html

In [23]:
# Python
import mlflow.prophet
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_components_plotly
from sklearn.metrics import mean_squared_error
from src.app.app_utils import plot_plotly_with_testset


## 데이터 불러오기

In [24]:
# Python
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df.head()

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


## 학습, 테스트셋 준비

In [25]:
df['ds'] = pd.to_datetime(df.ds)
df_train = df[df.ds < "2015-01-01"]
df_test = df[(df.ds >= "2015-01-01") & (df.ds < "2016-01-01")]

## 모델 학습 및 평가

In [27]:
experiment = mlflow.get_experiment_by_name("train_prophet_model")
if experiment is None:
    experiment_id = mlflow.create_experiment("train_prophet_model")
    print("Experiment train_prophet_model is created")
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(experiment_id=experiment_id) as run:
    model = Prophet()
    model.fit(df_train)

    future = model.make_future_dataframe(periods=365, include_history=False)
    forecast = model.predict(future)

    results = pd.merge(
        df_test[['ds','y']],
        forecast[['ds','yhat']],
        on='ds', how="left"
        )
    rmse_ = mean_squared_error(results.y, results.yhat)
    mlflow.log_metric("rmse", rmse_)
    mlflow.prophet.log_model(model, artifact_path="prophet_model")

    # 모델 등록
    mlflow.register_model(
            f"runs:/{run.info.run_id}/prophet_model", "prophet_model"
        )

01:48:10 - cmdstanpy - INFO - Chain [1] start processing
01:48:10 - cmdstanpy - INFO - Chain [1] done processing
/Users/jeanboy/workspace/time-series-ml-app/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning:

Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.

/Users/jeanboy/workspace/time-series-ml-app/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning:

Setuptools is replacing distutils.



## 시각화

In [20]:
future = model.make_future_dataframe(periods=365, include_history=True)
forecast = model.predict(future)
plot_plotly_with_testset(model, forecast, df_test)

In [21]:
plot_components_plotly(model, forecast)